# Builder

It allows constructing complex objects step by step.

It’s especially useful when you need to create an object with lots of possible configuration options.


Advantages of using Builder Method:
- Reusability: While making the various representations of the products, we can use the same construction code for other representations as well.
- Single Responsibility Principle: We can separate out both the business logic as well as the complex construction code from each other.
- Construction of the object: Here we construct our object step by step, defer construction steps or run steps recursively.

The main parts for Builder are only 2 components:
 - Product (order): represents the complex object under construction. Builder builds the product's internal representation and defines the process by which it's assembled
includes classes that define the constituent parts, including interfaces for assembling the parts into the final result
 - Builder (order builder) constructs and assembles parts of the product by implementing the Builder interface defines and keeps track of the representation it creates provides an interface for retrieving the product.

But normally, we need a client/director to use the builder to build the actual objects.

Builder Pattern is used when:
- The creation algorithm of a complex object is independent from the parts that actually compose the object.
- The system needs to allow different representations for the objects that are being built.

## Fluent Builder

This is to make the builder fluent so to build the object through a chaining process, instead of calling the constructor repeatedly.
It will returns a tree structure of objects to be built with a root object.

This can be usefule for situations like: file builder (html, json, xml...).

In [1]:
# sample program for order 

# order object
class Order:
    def __init__(self, name, quantity=1):
        self.name = name
        self.quantity = quantity
        self.orders = []

    def print_out(self):
        string = '\n'.join([f"ordered {out.quantity} times {out.name}." for out in self.orders])
        return string

# the builder for order
class OrderBuilder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.root_order = Order(root_name)
        
    def add_order(self, name, quantity):
        order = Order(name, quantity)
        self.root_order.orders.append(order)
        return self # make the builder fluent

    def print_out(self):
        string = f"for order {self.root_order.name}:\n" + self.root_order.print_out()
        return string

In [17]:
# instantiate order builder
order_builder = OrderBuilder("order11")
# add orders
order_builder.add_order("hamburger", 2).add_order("salad", 1).add_order("cake", 4)
# print all built orders
print(order_builder.print_out())

for order order11:
ordered 2 times hamburger.
ordered 1 times salad.
ordered 4 times cake.


## Communicating

In c++, this allows the to force the user to use the builder instead by hidinng the constrcutor in order to apply some constrution rules. In python, we can't hide the constructor, but we can still take advantage of this conviniency if we like.
So instead, we can set the constructor to default and use properties to do the construction (not in the below example) to enforce our builder rules.


In [39]:
class Order:
    def __init__(self, name, quantity=1):
        self.name = name
        self.quantity = quantity
        self.orders = []

    def print_out(self):
        string = '\n'.join([f"ordered {out.quantity} times {out.name}." for out in self.orders])
        return string

    @staticmethod
    def build(name):
        return OrderBuilder(name) # communicate with the builder

class OrderBuilder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.root_order = Order(root_name)
        
    def add_order(self, name, quantity):
        order = Order(name, quantity)
        self.root_order.orders.append(order)
        return self # make the builder fluent

    def print_out(self):
        string = f"for order {self.root_order.name}:\n" + self.root_order.print_out()
        return string

    def build(self):
        return self.root_order

    def __call__(self): # allow to return the Order object instead of builder
        return self.root_order
        

In [41]:
#return the builder object
order_builder = Order.build("order11").add_order("hamburger", 2).add_order("salad", 1).add_order("cake", 4)
print(type(order_builder))
# print all built orders
print(order_builder.print_out())

# return the Order object - need call
order = Order.build("order11").add_order("hamburger", 2).add_order("salad", 1).add_order("cake", 4)()
print(type(order))
# print all built orders
print(order.print_out())

<class '__main__.OrderBuilder'>
for order order11:
ordered 2 times hamburger.
ordered 1 times salad.
ordered 4 times cake.
<class '__main__.Order'>
ordered 2 times hamburger.
ordered 1 times salad.
ordered 4 times cake.


## Composite Builder

Sometimes, an object needs several builders to make the construction, with each targets one or some of the object's attributes.
Because an object can be dependent to several other objects. 
For the sake of the rule of single responsability, divide the builder into several dedicated builders seems the optimal solution.

In [74]:
# object
class Order:
    def __init__(self, name, quantity=1):
        self.name = name
        self.quantity = quantity
        self.orders = []

        self.address = None
        self.contact = None
        self.paiment = None
        self.price = 0

    def print_out(self):
        string = '\n'.join([f"ordered {out.quantity} times {out.name}." for out in self.orders]) + '\n'
        string += f"delivered to: {self.address}\n"
        string += f"with contact: {self.contact}\n"
        string += f"Paid by: {self.paiment} for ${self.price} "
        return string

    @staticmethod
    def build(name):
        return OrderBuilder(name)

# base builder
class OrderBuilderBase:
    def __init__(self, order:Order):
        self.root_order = order

    def __call__(self): # allow to return the Order object instead of builder
        return self.root_order

    # facets
    def delivered_to(self):
        return DeliveryBuilder(self.root_order)
    def paid_by(self):
        return paimentBuilder(self.root_order)

# concrete builder
class OrderBuilder(OrderBuilderBase):
    def __init__(self, root_name):
        super().__init__(Order(root_name))
        self.root_name = root_name
        
    def add_order(self, name, quantity):
        order = Order(name, quantity)
        self.root_order.orders.append(order)
        return self # make the builder fluent

    def print_out(self):
        string = f"for order {self.root_order.name}:\n" + self.root_order.print_out()
        return string

    def build(self):
        return self.root_order

# concrete builder
class DeliveryBuilder(OrderBuilderBase):
    def __init__(self, order):
        super().__init__(order)

    def at(self, address):
        self.root_order.address = address
        return self

    def phone(self, phone_type, phone_number):
        self.root_order.contact = (phone_type, phone_number)
        return self

# concrete builder
class paimentBuilder(OrderBuilderBase):
    def __init__(self, order):
        super().__init__(order)

    def by(self, pay):
        self.root_order.paiment = pay
        return self

    def worth(self, amount):
        self.root_order.price = amount
        return self
        

In [75]:
order = Order.build("order11").add_order("hamburger", 2)\
                              .add_order("salad", 1)\
                              .add_order("cake", 4)\
             .delivered_to().at("3403, rue bad H7D9L3")\
                            .phone("cell", "4872933489")\
             .paid_by().by("credit card")\
                       .worth(46)()
print(type(order))
print(order.print_out())

<class '__main__.Order'>
ordered 2 times hamburger.
ordered 1 times salad.
ordered 4 times cake.
delivered to: 3403, rue bad H7D9L3
with contact: ('cell', '4872933489')
Paid by: credit card for $46 


## Concrete Example - ElasticSearch

https://sergiiblog.com/python-flask-elasticsearch-front-controller-and-api-documentation/

In [39]:
# base object
import abc 

class BaseSearchCriteria:
    @abc.abstractmethod
    def print(self):
        pass

# concrete object
class HotelSearchCriteria(BaseSearchCriteria):
    
    DEFAULT_PAGE = 1
    DEFAULT_SIZE = 10
    SIZE_MIN = 1
    SIZE_MAX = 20
    
    def __init__(self):
        self._page = self.DEFAULT_PAGE
        self._size = self.DEFAULT_SIZE
        self._free_places_at_now = False
        self._hotel_name = None
        self._city_name = None
        self._hotel_age = None
        self._hotel_stars = None
        self._geo_coordinates = None

    @property
    def page(self):
        return self._page

    @page.setter
    def page(self, page):
        self._page = page

    @property
    def size(self):
        return self._size

    @size.setter
    def size(self, size):
        self._size = size

    @property
    def free_places_at_now(self):
        return self._free_places_at_now

    @free_places_at_now.setter
    def free_places_at_now(self, _free_places_at_now):
        self._free_places_at_now = _free_places_at_now

    @property
    def hotel_name(self):
        return self._hotel_name

    @hotel_name.setter
    def hotel_name(self, hotel_name):
        self._hotel_name = hotel_name

    @property
    def city_name(self):
        return self._city_name

    @city_name.setter
    def city_name(self, city_name):
        self._city_name = city_name

    @property
    def hotel_age(self):
        return self._hotel_age

    @hotel_age.setter
    def hotel_age(self, hotel_age):
        self._hotel_age = hotel_age

    @property
    def hotel_stars(self):
        return self._hotel_stars

    @hotel_stars.setter
    def hotel_stars(self, hotel_stars):
        self._hotel_age = hotel_stars

    @property
    def geo_coordinates(self):
        return self._geo_coordinates

    @geo_coordinates.setter
    def geo_coordinates(self, geo_coordinates):
        self._geo_coordinates = geo_coordinates

    def print(self):
        print(f"page: {self._page}\n \
                size: {self._size}\n \
                free_places_at_now : {self._free_places_at_now}\n \
                hotel_name: {self._hotel_name}\n \
                city_name: {self._city_name}\n \
                hotel_age: {self._hotel_age}\n \
                hotel_stars: {self._hotel_stars}\n \
                geo_coordinates: ({self._geo_coordinates.values()}")

In [30]:
PAGE = 'page'
SIZE = 'size'
HOTEL_NAME = "n"
HOTEL_CITY_NAME_EN = "c"
CITY_CENTER_LAT = "lat"
CITY_CENTER_LNG = "lng"
HOTEL_STARS = "stars"
HOTEL_FREE_PLACES = "fpn"
HOTEL_AGE = "age"

# base criteria builder
class BaseCriteriaBuilder(abc.ABC):

    @abc.abstractmethod
    def get_criteria(self):
        pass

    @abc.abstractmethod
    def create_criteria(self):
        pass

# concrete criteria builder
class HotelSearchCriteriaUrlBuilder(BaseCriteriaBuilder):
    def __init__(self, data: dict):
        self.data = data
        self.criteria = HotelSearchCriteria()

    def create_criteria(self):
        if PAGE in self.data and isinstance(self.data[PAGE], int):
            self.criteria.page = self.data[PAGE]

        if SIZE in self.data and isinstance(self.data[SIZE], int) \
                and self.data[SIZE] <= HotelSearchCriteria.SIZE_MAX:
            self.criteria.size = self.data[SIZE]

        if HOTEL_AGE in self.data and isinstance(self.data[HOTEL_AGE], int):
            self.criteria.hotel_age = self.data[HOTEL_AGE]

        if HOTEL_STARS in self.data and isinstance(self.data[HOTEL_STARS], int):
            self.criteria.hotel_stars = self.data[HOTEL_STARS]

        if HOTEL_NAME in self.data and isinstance(self.data[HOTEL_NAME], str):
            self.criteria.hotel_name = self.data[HOTEL_NAME]

        if HOTEL_CITY_NAME_EN in self.data \
                and isinstance(self.data[HOTEL_CITY_NAME_EN], str):
            self.criteria.city_name = self.data[HOTEL_CITY_NAME_EN]

        if HOTEL_FREE_PLACES in self.data \
                and isinstance(self.data[HOTEL_FREE_PLACES], bool):
            self.criteria.free_places_at_now = self.data[HOTEL_FREE_PLACES]

        if (
                CITY_CENTER_LAT in self.data
                and isinstance(self.data[CITY_CENTER_LAT], float)
                and CITY_CENTER_LNG in self.data
                and isinstance(self.data[CITY_CENTER_LNG], float)
        ):
            self.criteria.geo_coordinates = {
                "lat": self.data[CITY_CENTER_LAT],
                "lon": self.data[CITY_CENTER_LNG]
            }

    def get_criteria(self) -> HotelSearchCriteria:
        return self.criteria

In [12]:
# base director
class BaseCriteriaDirector(abc.ABC):
    @abc.abstractmethod
    def build_criteria(self):
        pass

    @abc.abstractmethod
    def get_criteria(self):
        pass

    @abc.abstractmethod
    def print_criteria(self):
        pass

# concrete director
class HotelSearchCriteriaDirector(BaseCriteriaDirector):
    def __init__(self, builder: BaseCriteriaBuilder):
        self.builder = builder

    def build_criteria(self):
        self.builder.create_criteria()

    def get_criteria(self) -> HotelSearchCriteria:
        return self.builder.get_criteria()

    def print_criteria(self):
        self.builder.get_criteria().print()

In [40]:
# usage
data = dict(page = 3,\
            size = 4,\
            n =  "Translander",\
            c = "Montreal",\
            lat = 45.0,\
            lng = 56.9,\
            stars = 4,\
            fpn = 450,\
            age = 15)
builder = HotelSearchCriteriaUrlBuilder(data)
director = HotelSearchCriteriaDirector(builder)
director.build_criteria()
director.print_criteria()

page: 3
                 size: 4
                 free_places_at_now : False
                 hotel_name: Translander
                 city_name: Montreal
                 hotel_age: 4
                 hotel_stars: None
                 geo_coordinates: (dict_values([45.0, 56.9])
